In [7]:
import numpy as np
import os
import glog as log
import copy

import open3d as o3d
#from __future__ import division
from open3d import JVisualizer

import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

In [48]:
# FILL PATHS BELOW
#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Downloads/tesse_semantics_2.ply"

#gt_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"

#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/tesse_semantics_3.ply"

gt_mesh_path = "/home/tonirv/Documents/uHumans2_VIO/subway_scene/uHumans2_subway_s1_00h/mesh_gt.ply"
est_mesh_path = "/home/tonirv/Documents/uHumans2_VIO/subway_scene/uHumans2_subway_s1_00h/mesh_gt.ply"
semantic_csv_path = '/home/tonirv/datasets/uHumans2/apartment_scene/tesse_multiscene_archviz1_segmentation_mapping.csv'

In [20]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 487162 points and 501621 triangles.
[[-4.725     -6.375     -0.0361858]
 [-4.775     -6.425     -0.0318488]
 [-4.725     -6.425     -0.0277364]
 ...
 [-8.30093    0.425      1.675    ]
 [-8.325      0.425      1.70417  ]
 [-8.325      0.400931   1.675    ]]
[[     0      1      2]
 [     3      1      0]
 [     4      5      6]
 ...
 [487157 487153 487156]
 [487158 487156 487155]
 [487159 487160 487161]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 487162 points and 501621 triangles.
[[-4.725     -6.375     -0.0361858]
 [-4.775     -6.425     -0.0318488]
 [-4.725     -6.425     -0.0277364]
 ...
 [-8.30093    0.425      1.675    ]
 [-8.325      0.425      1.70417  ]
 [-8.325      0.400931   1.675    ]]
[[     0      1      2]
 [     3      1      0]
 [     4      5      6]
 ...
 [487157 487153 487156]
 [487158 487156 487155]
 [487159 487160 487161]]



In [21]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [33]:
################ VIT ######################
# Align Pointclouds Manually:
#est_mesh.mesh_o3d.translate([0, -5, 0])
#gt_mesh.transform_left(righthand_R_lefthand)
gt_mesh.transform_left(enu_R_unity)

Transforming mesh according to left matrix:
[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [23]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

In [24]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [25]:
# Calculate normals for nice visualization
# THIS COLORS THE PCL?>>>>>????
#o3d.geometry.estimate_normals(
#        est_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))
#o3d.geometry.estimate_normals(
#        gt_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))

In [26]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [27]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [28]:
# ICP params
ICP_THRESHOLD = 1.5
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [29]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [30]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [31]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 0.088872, inlier_rmse = 0.858367, and correspondence_set size of 43295
Access transformation to get result.


In [32]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [34]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 0.987760, inlier_rmse = 0.483553, and correspondence_set size of 481199
Access transformation to get result.

Transformation is:
[[ 0.97025601  0.05537522  0.23566258 -0.7847284 ]
 [ 0.24056675 -0.32927179 -0.91307597  6.10193699]
 [ 0.02703526  0.94261003 -0.33279937  0.83131788]
 [ 0.          0.          0.          1.        ]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 481199 elements.
Use numpy.asarray() to access data.



In [35]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [36]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [37]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [38]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    for correspondence in correspondences:
        if np.allclose(est_pcl.colors[correspondence[0]],
                       gt_pcl.colors[correspondence[1]]):
            total_positive_matches += 1
        else:
            total_negative_matches += 1

    print("Positive color matches: ",total_positive_matches)
    print("Negative color matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(total_positive_matches / total_correspondences * 100))
    print ("Negative: {}  % ".format(total_negative_matches / total_correspondences * 100))

In [39]:
calc_corresp(est_pcl, gt_pcl, correspondences)

Positive color matches:  211421
Negative color matches:  269778
Total correspondences:  481199
Positive: 43.93629246943572  % 
Negative: 56.06370753056428  % 


In [41]:
print(est_pcl.colors[0])

[0.02745098 0.31372549 0.21960784]


In [57]:
# Import Semantic Labels
df = pd.read_csv(semantic_csv_path)
df

,id,red,green,blue,alpha
0,Graphics,244,74,215,255
1,Mirror_Floor,225,121,28,255
2,Books,15,18,194,255
3,Strala,117,122,95,255
4,Strala_Bulb,212,90,129,255
5,Strala_Cable,228,148,186,255
6,Kragsta_Short,205,158,166,255
7,Props_Vase2,34,32,78,255
8,Scatter_Rocks_Grey_Sharp_Rough_MRGSR3452_3,250,74,32,255
9,Sinnersro_Short,41,51,214,255


In [58]:
normalized_df = copy.deepcopy(df)
normalized_df['normalized_red'] = df['red'] / 255
normalized_df['normalized_green'] = df['green'] / 255
normalized_df['normalized_blue'] = df['blue'] / 255

print(sum(np.isclose(normalized_df['normalized_red'], gt_pcl.colors[0][0])))


6


In [59]:
from hashlib import sha1

from numpy import all, array, uint8


class hashable(object):
    r'''Hashable wrapper for ndarray objects.
        Instances of ndarray are not hashable, meaning they cannot be added to
        sets, nor used as keys in dictionaries. This is by design - ndarray
        objects are mutable, and therefore cannot reliably implement the
        __hash__() method.
        The hashable class allows a way around this limitation. It implements
        the required methods for hashable objects in terms of an encapsulated
        ndarray object. This can be either a copied instance (which is safer)
        or the original object (which requires the user to be careful enough
        not to modify it).
    '''
    def __init__(self, wrapped, tight=False):
        r'''Creates a new hashable object encapsulating an ndarray.
            wrapped
                The wrapped ndarray.
            tight
                Optional. If True, a copy of the input ndaray is created.
                Defaults to False.
        '''
        self.__tight = tight
        self.__wrapped = array(wrapped) if tight else wrapped
        self.__hash = int(sha1(wrapped.view(uint8)).hexdigest(), 16)

    def __eq__(self, other):
        return all(self.__wrapped == other.__wrapped)

    def __hash__(self):
        return self.__hash

    def unwrap(self):
        r'''Returns the encapsulated ndarray.
            If the wrapper is "tight", a copy of the encapsulated ndarray is
            returned. Otherwise, the encapsulated ndarray itself is returned.
        '''
        if self.__tight:
            return array(self.__wrapped)



In [60]:
# No need to hash, we have ids now
red_col = df['red'].to_numpy()
blue_col = df['blue'].to_numpy()
green_col = df['green'].to_numpy()

def f(x):    
    return str(x['normalized_red'])+str(x['normalized_green'])+str(x['normalized_blue'])

hashed_df = copy.deepcopy(normalized_df)
hashed_df['hash'] = hashed_df.apply(f, axis=1)
hashed_df

,id,red,green,blue,alpha,normalized_red,normalized_green,normalized_blue,hash
0,Graphics,244,74,215,255,0.956863,0.290196,0.843137,0.95686274509803930.29019607843137260.84313725...
1,Mirror_Floor,225,121,28,255,0.882353,0.474510,0.109804,0.88235294117647060.47450980392156860.10980392...
2,Books,15,18,194,255,0.058824,0.070588,0.760784,0.0588235294117647050.070588235294117650.76078...
3,Strala,117,122,95,255,0.458824,0.478431,0.372549,0.45882352941176470.478431372549019630.3725490...
4,Strala_Bulb,212,90,129,255,0.831373,0.352941,0.505882,0.83137254901960790.352941176470588260.5058823...
5,Strala_Cable,228,148,186,255,0.894118,0.580392,0.729412,0.89411764705882360.58039215686274510.72941176...
6,Kragsta_Short,205,158,166,255,0.803922,0.619608,0.650980,0.8039215686274510.61960784313725490.650980392...
7,Props_Vase2,34,32,78,255,0.133333,0.125490,0.305882,0.133333333333333330.125490196078431370.305882...
8,Scatter_Rocks_Grey_Sharp_Rough_MRGSR3452_3,250,74,32,255,0.980392,0.290196,0.125490,0.98039215686274510.29019607843137260.12549019...
9,Sinnersro_Short,41,51,214,255,0.160784,0.200000,0.839216,0.16078431372549020.20.8392156862745098


In [61]:
# Generate table from color to id.
def label_from_color(color):
    norm_r = color[0]
    norm_g = color[1]
    norm_b = color[2]
    # TODO(Toni): can be greatly optimized... TO SLOW NOW
    # TODO(Toni): you are comparing floats with == ......
    return normalized_df.loc[(normalized_df['normalized_red'] == norm_r) & 
                  (normalized_df['normalized_green'] == norm_g) & 
                  (normalized_df['normalized_blue'] == norm_b)]['id'].unique().tolist()[0]

In [62]:
# Compare labels between correspondences:
total_label_correspondences = {i:0 for i in normalized_df['id'].unique()}
# Initialize dictionaries to 0:
total_positive_matches = {i:0 for i in normalized_df['id'].unique()}
total_negative_matches = copy.deepcopy(total_positive_matches)

In [63]:
def calc_per_label_corresp():
    for correspondence in correspondences:
        est_pcl_color = est_pcl.colors[correspondence[0]]
        color_hash = str(est_pcl_color[0])+str(est_pcl_color[1])+str(est_pcl_color[2])
        label_id = label_from_color(est_pcl_color)
        if np.allclose(est_pcl_color, gt_pcl.colors[correspondence[1]]):
            total_positive_matches[label_id] += 1
        else:
            total_negative_matches[label_id] += 1

In [64]:
calc_per_label_corresp()

In [65]:
print "Dict for total_positive_matches: \n", total_positive_matches
print "Dict for total_negative_matches: \n", total_negative_matches
for label_id in df['id'].unique().tolist():
    total_label_correspondences[label_id] = (total_positive_matches[label_id] + total_negative_matches[label_id])
print "Dict for total_label_correspondences: \n", total_label_correspondences
for label_id in df['id'].unique().tolist():
    print ("ID: {}".format(label_id))
    print ("Positive: {}  % ".format(total_positive_matches[label_id] / total_label_correspondences[label_id] * 100))
    print ("Negative: {}  % ".format(total_negative_matches[label_id] / total_label_correspondences[label_id] * 100))

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-65-2c6f17b74a08>, line 1)